In [ ]:
#get_file_word.py
#!/usr/bin/env python
# coding: utf-8

In [ ]:
# metodo para ler arquivo word



In [2]:
# importar biblioteca
import os
# montagem de diretório na rede a partir de um diretório local (pastalocal)
#mkdir pastaLocal
dir_destino= "pastaLocal"
if not os.path.exists(dir_destino):
    os.makedirs(dir_destino)

In [43]:
print("O diretório "+dir_destino+" exite?",os.path.exists(dir_destino))

O diretório pastaLocal exite? True


In [ ]:
# achando o caminho dentro da rede - MONTANDO DIRETÓRIO NA REDE
!mount_smbfs smb://ensi-filer02/gerpubprop$ pastaLocal/
#!umount pastaLocal/

##### se der ERRO nesse comando ejetar a pasta gerpubprop$ antes e rodar o codigo novamente

In [3]:
#TESTANDO O MONTAGEM DO DIRETÓRIO
os.listdir('/Users/wpessoa/repositorios/portifolio_nge/pastaLocal/Coordenacao de Gestao Editorial/2021/TRIAGEM/')

['FEITO',
 'ID233867-06_12_2019_EFEITOS DA GESTÃO SOBRE A PRODUTIVIDADE - PARA PUBLICAÇÃO_Final_19122019_155125.docx']

In [76]:
import get_file_word as gf
import datetime
import pandas as pd
from datetime import date
import shutil

data_atual = date.today()

#setar variáveis onde estão arquivos
fileExt = r".docx"
fileDir = r"/Users/wpessoa/repositorios/portifolio_nge/pastaLocal/Coordenacao de Gestao Editorial/2021/TRIAGEM/"

# verificar se existe o diretório
print("Existe esse diretório? ",os.path.exists(fileDir)) 

#criar variável contendo todos os arquivos com extensão DOCX 
files_array = [_ for _ in os.listdir(fileDir) if _.endswith(fileExt)]

print("Quantidade de arquivos do word no diretório: ",len(files_array))
print("-------------------------------------------------------------")

# rodar método get_info para ler aquivos do Word e retornar features de cada arquivo
# argumentos (diretório e array dos arquivos word)
# retorna (NºID,Qtd_caracteres,Qtd_tabela,Qtd_image,data)

var=gf.get_info(fileDir,files_array)

# gerar um Dataframe

df = pd.DataFrame(var,columns=['Nun_ID',"Qtd_PG_word","Qtd_carac","Qtd_tabela","Qtd_image","Qtd_estilos", "Data"])

# recuperar conteudo do arquivo Excel
excelFile_old = fileDir+"pre-triagem"+"-"+str(data_atual.year)+".xlsx"
# marca nessa variável se não existe excel gravado

if os.path.exists(excelFile_old):
    excelFile = pd.read_excel(excelFile_old)
    file_type = type(excelFile)

# OUTPUT das variáveis do método
for i in range(0,len(df.Nun_ID)):
    print(files_array[i])
    print("Número de ID: ",df.Nun_ID[i])
    print("Qtd de páginas: ",df.Qtd_PG_word[i])
    print("Qtd de caracteres: ",df.Qtd_carac[i])
    print("Qtd de tabelas: ",df.Qtd_tabela[i])
    print("Qtd de imagens: ",df.Qtd_image[i])
    print("Qtd de estilos no Word: ",df.Qtd_estilos[i])
    print("Data de criação do documento: ",df.Data[i])
    print("-------------------------------------------------------------")
    #df.loc[[i]].to_excel("/Users/wpessoa/repositorios/portifolio_nge/pastaLocal/Coordenacao de Gestao Editorial/2021/TRIAGEM/pre-triagem-ID"+df.Nun_ID[i]+".xlsx",index=False,header=True )


if os.path.exists(excelFile_old):
    
# gerar arquivo excel para arquivo tratado
df.to_excel(fileDir+"pre-triagem"+"-"+str(data_atual.year)+".xlsx",index=False,header=True )

# copiara os arquivo word para pasta de processo feito
dir_destino = fileDir+"/FEITO/"
if len(files_array)>0:
    try:
        for linha in files_array:
            if not os.path.exists(dir_destino):
                os.makedirs(dir_destino)
            shutil.move(fileDir+ linha, dir_destino + linha)
    except OSError:
            print("Oops!  erro no método copiara arquivo - ERRO NO S.O.")

Existe esse diretório?  True
Quantidade de arquivos do word no diretório:  2
-------------------------------------------------------------
ID236690-Novas barreiras e tendências no comércio internacional.docx
Número de ID:  236690
Qtd de páginas:  49
Qtd de caracteres:  80889
Qtd de tabelas:  6
Qtd de imagens:  12
Qtd de estilos no Word:  26
Data de criação do documento:  2021-01-25
-------------------------------------------------------------
ID233867-06_12_2019_EFEITOS DA GESTÃO SOBRE A PRODUTIVIDADE - PARA PUBLICAÇÃO_Final_19122019_155125.docx
Número de ID:  233867
Qtd de páginas:  31
Qtd de caracteres:  58108
Qtd de tabelas:  9
Qtd de imagens:  23
Qtd de estilos no Word:  17
Data de criação do documento:  2019-12-06
-------------------------------------------------------------


In [78]:
excelFile_old = fileDir+"pre-triagem"+"-"+str(data_atual.year)+".xlsx"
excelFile = pd.read_excel(excelFile_old)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/wpessoa/repositorios/portifolio_nge/pastaLocal/Coordenacao de Gestao Editorial/2021/TRIAGEM/pre-triagem-2021.xlsx'

In [63]:
df.loc[[0]]

,Nun_ID,Qtd_PG_word,Qtd_carac,Qtd_tabela,Qtd_image,Qtd_estilos,Data
0,236690,49,80889,6,12,26,2021-01-25


In [70]:

print(str(data_atual.year))

2021
